# Baseline solution

<!--- @wandbcode{course-lesson1} -->

In this notebook we will create a baseline solution to our semantic segmentation problem. To iterate fast a notebook is a handy solution. We will then refactor this code into a script to be able to use hyperparameter sweeps.

In [1]:
import sys
sys.path.insert(0, 'C:\\Users\\Griffin\\Documents\\GitHub\\wandb-edu')

In [2]:
import torch
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from types import SimpleNamespace
from pathlib import Path
import json
import params

import wandb
import rave
import rave.dataset
import rave.blocks
import rave.pqmf

C:\Users\Griffin\Anaconda3\envs\wandb_edu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Again, we're importing some global configuration parameters from `params.py` file. We have also defined some helper functions in `utils.py` - for example metrics we will track during our experiments.

Let's now create a `train_config` that we'll pass to W&B `run` to control training hyperparameters. 

In [3]:
train_config = SimpleNamespace(
    framework="RAVEV2Encoder",
    batch_size=8,
    epochs=10, 
    seed=42,
    sampling_rate=16000,
    n_signal=64000,
    n_band=16,
    attenuation=100,
    latent_size=10,
    n_out=2,
    lr=2e-3,
    kernel_size=3,
    dilations=[1, 3, 9],
    ratios=[4, 4, 4, 2],
    capacity=96
)

We are setting seed for reproducibility. 

In [4]:
seed_everything(train_config.seed)

Global seed set to 42


42

In [5]:
wandb_logger = WandbLogger(project=params.WANDB_PROJECT, config=train_config)

wandb: Currently logged in as: gcpage. Use `wandb login --relogin` to force relogin


As usual, we will use W&B Artifacts to track the lineage of our models. 

In [6]:
data_at = wandb_logger.use_artifact(f'{params.RAW_DATA_AT}:latest')
#processed_dataset_dir = Path(processed_data_at.download())
#df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

In [7]:
data_path = Path('C:/Users/Griffin/Documents/datasets/nsynth/nsynth-valid')
db_path = data_path / 'processed-labeled'
num_workers = 0

In [8]:
config = wandb.config
config.dilations

[1, 3, 9]

In [9]:
dataset = rave.dataset.get_dataset(str(db_path), config.sampling_rate, config.n_signal)
signal, label = dataset[0]
len(signal)

64000

In [10]:
train, val = rave.dataset.split_dataset(dataset, 80)

train set: 10142 examples
val set: 2536 examples


In [11]:
train_loader = DataLoader(train,
                   train_config.batch_size,
                   True,
                   drop_last=True,
                   num_workers=num_workers)

val_loader = DataLoader(val,
                 train_config.batch_size,
                 False,
                 num_workers=num_workers)


We are using `wandb.config` to track our training hyperparameters. 

We will use *intersection over union* metrics: mean across all classes (MIOU) and IOU for each class separately. Our model will be a `unet` based on pretrained `resnet18` backbone. 

In [12]:
pqmf = rave.pqmf.CachedPQMF(config.attenuation, config.n_band)

In [13]:
model = rave.model.EncoderV2Timbre(data_size=config.n_band,
                                   n_out=config.n_out,
                                   latent_size=config.latent_size,
                                   sampling_rate=config.sampling_rate,
                                   lr=config.lr,
                                   kernel_size=config.kernel_size,
                                   dilations=config.dilations,
                                   ratios=config.ratios,
                                   capacity=config.capacity,
                                   pqmf=pqmf)
                                   

In `fastai` we already have a callback that integrates tightly with W&B, we only need to pass the `WandbCallback` to the learner and we are ready to go. The callback will log all the useful variables for us. For example, whatever metric we pass to the learner will be tracked by the callback.

In [14]:
callbacks = [ModelCheckpoint(monitor="val_accuracy", mode="max")]
trainer = Trainer(max_epochs=config.epochs,
                  logger=wandb_logger,
                  callbacks=callbacks)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Let's train our model!

In [15]:
trainer.fit(model, train_loader, val_loader)


  | Name    | Type       | Params
---------------------------------------
0 | pqmf    | CachedPQMF | 16.7 K
1 | encoder | EncoderV2  | 17.3 M
---------------------------------------
17.3 M    Trainable params
0         Non-trainable params
17.3 M    Total params
69.387    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\Griffin\Anaconda3\envs\wandb_edu\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

AttributeError: 'EncoderV2Timbre' object has no attribute 'loss'

We will log a table with model predictions and ground truth to W&B, so that we can do error analysis in the W&B dashboard. 

In [ ]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({"pred_table":table})

We are reloading the model from the best checkpoint at the end and saving it. To make sure we track the final metrics correctly, we will validate the model again and save the final loss and metrics to `wandb.summary`. 

In [ ]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v

In [ ]:
wandb.finish()